# Initiale Datevorbereitung

## Ziel

Die Rohdaten sollen bereinigt und in Training- und Test-Set gespalten werden

Das beinhaltet:

* Daten in Reihenfolge bringen
* Fehlerhafte Features entfernen
* Design einer sinnvollen Zielvariable / Variationen von Zielvariablen
* Zusammenfügen der Datensätze
* Daten randomisieren
* Aufteilen in Training- und Test-Set


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from datetime import datetime, timedelta, timezone

In [ ]:
plt.rcParams["figure.figsize"] = (50,40)

In [ ]:
failures_2016 = pd.read_csv("./data/init/failures-2016.csv", sep=";")
failures_2017 = pd.read_csv("./data/init/failures-2017.csv", sep=";")
metmast_2016 = pd.read_csv("./data/init/metmast-2016.csv", sep=";")
metmast_2017 = pd.read_csv("./data/init/metmast-2017.csv", sep=";")
signals_2016 = pd.read_csv("./data/init/signals-2016.csv", sep=";")
signals_2017 = pd.read_csv("./data/init/signals-2017.csv", sep=";")

## 1. Signale betrachten

### Signale beider Jahre kombinieren

In [ ]:
signals = pd.concat([signals_2016, signals_2017])

In [ ]:
signals.describe()

### Die Signale für jede Turbine aufbereiten und plotten

In [ ]:
turbine_names = signals["Turbine_ID"].unique()

In [ ]:
def create_df_for_each_turbine(signals: pd.DataFrame) -> list[pd.DataFrame]:
    turbine_dfs = list();

    for turbine in turbine_names:
        test = signals["Turbine_ID"]
        turbine_df = signals[signals["Turbine_ID"] == turbine]
        turbine_df = turbine_df.sort_values("Timestamp")
        turbine_df = turbine_df.reset_index()
        turbine_dfs.append(turbine_df)

    return turbine_dfs

turbine_dfs = create_df_for_each_turbine(signals)


In [ ]:
def plot_data(df: pd.DataFrame, title: str, n_agg: int, figsize=(10,9)) -> None:
    my_df = df.groupby(df.index // n_agg).agg("mean")
    fig, axs = plt.subplots(figsize[0], figsize[1])
    fig.suptitle(title)
    fig.tight_layout(pad=5.0)
    ctr2 = -1
    for i, col in enumerate(my_df):
        mod = i % figsize[0]
        if mod == 0: ctr2 +=1
        ax = axs[mod, ctr2]
        ax.scatter(x=my_df.index, y=my_df[col], s=1)
        ax.set_title(col)

In [ ]:
# for i, df in enumerate(turbine_dfs):
    # plot_data(df, turbine_names[i], 20)

### Signale mit niedriger Varianz herausfiltern

In [ ]:

def get_signals_with_low_variance(df: pd.DataFrame, threshold=0) -> list:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    cont_data = df.select_dtypes(include=numerics)
    selector = VarianceThreshold(threshold=threshold)
    selector.fit(cont_data)
    inverted_list = ~np.array(selector.get_support())
    return cont_data.columns[inverted_list].tolist()

In [ ]:
cols_to_drop = set()
for i, df in enumerate(turbine_dfs):
    cols_to_drop.update(get_signals_with_low_variance(df))

### Signale wieder zusammenführen

In [ ]:
signals_clean = pd.concat(turbine_dfs)
signals_clean = signals_clean.drop(list(cols_to_drop), axis=1)

In [ ]:
signals_clean.describe()

## 2. Mast-Daten betrachten

In [ ]:
metmast = pd.concat([metmast_2016, metmast_2017])

In [ ]:
metmast.head()

In [ ]:
df = metmast.describe()
df.loc['dtype'] = metmast.dtypes
df.loc['size'] = len(metmast)
df.loc['n_NaN'] = metmast.isnull().mean() * len(metmast)

print(df)

In [ ]:
metmast = metmast.sort_values("Timestamp")
metmast = metmast.reset_index()

plot_data(metmast, "Mast-Daten", 10, (6, 7))

In [ ]:
cols_to_drop = get_signals_with_low_variance(metmast)

In [ ]:
# clean automatic features
metmast_clean = metmast.drop(cols_to_drop, axis=1)
# clean broken sensor data
metmast_clean = metmast_clean.drop(["Min_Winddirection2", "Max_Winddirection2", "Avg_Winddirection2", "Var_Winddirection2"], axis=1)

In [ ]:
metmast_clean.describe()

In [ ]:
failures = pd.concat([failures_2016, failures_2017])

In [ ]:
failures.head()

In [ ]:
failures.describe()

# Datensätze in einen Datensatz zusammenführen
## Sensor- und Mastdaten mergen

In [ ]:
merged_df = pd.merge(signals_clean.reset_index(drop=True), metmast_clean.reset_index(drop=True), on="Timestamp", how="left")

In [ ]:
merged_df.describe(include="all")

In [ ]:
# for column in merged_df.columns:
#     if (pd.api.types.is_numeric_dtype(merged_df[column])):
#         merged_df[column].fillna(merged_df[column].median(), inplace = True)

In [ ]:
merged_df.drop(columns=["index_x"], inplace=True)

In [ ]:
merged_df.sample(5)

## Targets vorbereiten und mergen

In [ ]:
failures_gearbox = failures[failures["Component"] == "HYDRAULIC_GROUP"]
failures_gearbox.reset_index(drop=True, inplace=True); failures_gearbox

Da die labels nicht auf 10 Minuten gerundet sind (Wie Sensor- und Mastdaten), wird das an dieser Stelle getan damit die Labels mit den restlichen Daten kompatibel sind:

In [ ]:
def get_round_minute_diff(datetime_in: datetime) -> timedelta:
    min = datetime_in.minute
    rounded_min = round(min, -1)
    diff = rounded_min - min
    return timedelta(minutes=diff)

In [ ]:
def convert_round_minute_to_time(datetime_in: datetime) -> datetime:
    td = get_round_minute_diff(datetime_in)
    return datetime_in + td

Für die Labels, werden 5 Stufen eingeführt, die alle 30 Tage um 1 erhöht werden. 
<br/>
<span style="color:red">TODO: Aktuell wird im Code nicht berücksichtigt, wenn sich labels überschneiden, da das für die Gearbox nicht der Fall ist.</span>

In [ ]:
def create_failure_list(classes: list[str], days_per_class: int, target_name: str) -> pd.DataFrame:
    days_lookback = len(classes) * days_per_class
    ten_mins_of_n_days = int(24 * 60 * days_lookback / 10)
    failure_list = []
    for i, failure in enumerate(failures_gearbox):
        turbine_id = str(failures_gearbox["Turbine_ID"][i])
        failure_ts = str(failures_gearbox["Timestamp"][i])
        failure_datetime = datetime.fromisoformat(failure_ts)
        rounded_datetime = convert_round_minute_to_time(failure_datetime)
        for iterator, current_class in enumerate(classes):
            for j in range(ten_mins_of_n_days):
                delta = timedelta(minutes=j*10)
                # Prüfen ob obere und untere Schranke passen.
                is_in_class = delta >= timedelta(days=iterator*days_per_class) and delta < timedelta(days=(iterator+1) * days_per_class)
                if (is_in_class):
                    new_datetime = rounded_datetime - delta
                    datetime_formated = new_datetime.replace(tzinfo=timezone.utc)
                    failure_list.append([turbine_id, datetime_formated.isoformat(), current_class])
    
    failure_df = pd.DataFrame(failure_list, columns=["Turbine_ID", "Timestamp", target_name])

    return failure_df


In [ ]:
class_target_name = "Risk Level"
risk_levels = ["low", "low-med", "medium", "med-high", "high"]
days_per_class = 30

failure_df = create_failure_list(classes=risk_levels, days_per_class=days_per_class, target_name=class_target_name)

Der Feature-Datensatz wird mit den Labels zusammengeführt. Dabei ist besonders wichtig, dass der Bezug zu der jeweiligen Turbine bestehen bleibt.

In [ ]:
labeled_df = pd.merge(merged_df.reset_index(drop=True), failure_df.reset_index(drop=True), on=["Turbine_ID", "Timestamp"], how="left"); 

In [ ]:
labeled_df[class_target_name].fillna(risk_levels[0], inplace = True)

In [ ]:
labeled_df.describe(include="all")

In [ ]:
labeled_df[class_target_name].value_counts()

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

# Daten in Training und Test Set splitten

In [ ]:
# Alle Daten ab August 2017 liegen im Testset
split_criterion_reg = labeled_df["Timestamp"] >= "2017-04-00T00:00:00+00:00"

test_gearbox = labeled_df[split_criterion_reg].reset_index(drop=True)
train_gearbox = labeled_df[~split_criterion_reg].reset_index(drop=True)

In [ ]:

plt.scatter(x=train_gearbox.index, y=train_gearbox[class_target_name])
# train_simple_regression_gearbox.describe()

# Erzeugte Daten abspeichern

In [ ]:
import os
os.makedirs("./data/multi", exist_ok=True)

In [ ]:
train_gearbox.to_csv("./data/multi/train_hydraulic.csv")
test_gearbox.to_csv("./data/multi/test_hydraulic.csv")